In [1]:
##################################################
## Binance Python Bot :]
##################################################

import time
# pip install python-binance
from binance.client import Client
from binance.enums import *
# pip install pandas
import pandas as pd
# pip install numpy
import numpy as np
import datetime as dt
import requests

In [2]:
# Functions
def computeRSI (data, time_window):
    diff = np.diff(data)
    up_chg = 0 * diff
    down_chg = 0 * diff
    
    # up change is equal to the positive difference, otherwise equal to zero
    up_chg[diff > 0] = diff[ diff>0 ]
    
    # down change is equal to negative deifference, otherwise equal to zero
    down_chg[diff < 0] = diff[ diff < 0 ]

    up_chg = pd.DataFrame(up_chg)
    down_chg = pd.DataFrame(down_chg)
    
    up_chg_avg   = up_chg.ewm(com=time_window-1 , min_periods=time_window).mean()
    down_chg_avg = down_chg.ewm(com=time_window-1 , min_periods=time_window).mean()
    
    rs = abs(up_chg_avg/down_chg_avg)
    rsi = 100 - 100/(1+rs)
    rsi = int(rsi[0].iloc[-1])
    return rsi

In [3]:
def bollingerband(symbol,width,intervalunit,length):
    if intervalunit=='1T':
        start_str='100 minutes ago UTC'
        interval_data='1m'
        
        D=pd.DataFrame(client.get_historical_klines(symbol=symbol,start_str=start_str,interval=interval_data))
        D.columns=['open_time','open','high','low','close','volume','close_time','qav','num_trades','taker_base_vol','taker_quote_vol','is_best_match']
        D['open_date_time']=[dt.datetime.fromtimestamp(x/1000) for x in D.open_time]
        D['symbol']=symbol
        D=D[['symbol','open_date_time','open','high','low','close','volume','num_trades','taker_base_vol','taker_quote_vol']]
        
        df=D.set_index("open_date_time")
        
        df['close']=df['close'].astype(float)
        
        df=df['close']
        df1=df.resample(intervalunit).agg({
            "close":"last"
                        
        })
        
        unit =width
        band1=unit*np.std(df1['close'][len(df1)-length:len(df1)])
        bb_center=np.mean(df1['close'][len(df1)-length:len(df1)])
        band_high=bb_center+band1
        band_low=bb_center-band1
    return band_high,bb_center,band_low

In [4]:
#definecoin
trdPair1 = 'BTT'
trdPair2 = 'USDT'
#bollingerband length and width
length=20
width=2

In [5]:
listofsellcoins=["ARPA"]

In [6]:
# Console header
print('___DATE______TIME_____BALANCE___RSI____PRICE____STRATEGY___TARGET-PRICE__')

___DATE______TIME_____BALANCE___RSI____PRICE____STRATEGY___TARGET-PRICE__


In [7]:
# Main loop
while True:
                   
       
        for i in range(len(listofsellcoins)):
            try:
                print(listofsellcoins[i])
                trdPair1=listofsellcoins[i]
                # Authenticate to Binance
                api_key = ''
                api_secret = ''
                winRate = 1.03
                client = Client(api_key, api_secret)

                # Initial values
                tradePair = trdPair1 + trdPair2
                price = client.get_ticker(symbol=tradePair)
                #print(price)
                btcCount = client.get_asset_balance(asset = trdPair1)
                btcCount = float(btcCount['free'])*float(price['askPrice'])
                USDTCount = client.get_asset_balance(asset = trdPair2)
                USDTCount = float(USDTCount['free'])

                # Find last trade
                if btcCount > USDTCount:
                    lastrade = trdPair1
                elif btcCount < USDTCount:
                    lastrade = trdPair2

                # Find last price
                trades = client.get_my_trades(symbol=tradePair)
                trades = trades[len(trades)-1]
                lasprice = float(trades['price'])
                print(lasprice)
                klines = client.get_klines(symbol=tradePair, interval='1m', limit='500')
                klines2 = client.get_historical_klines(tradePair, Client.KLINE_INTERVAL_1MINUTE, "1 day ago UTC")
                close = [float(entry[4]) for entry in klines]
                close_array = np.asarray(close)
                close_finished = close_array[:-1]
                rsi = computeRSI (close_finished, 14)
                bb_1m=bollingerband(tradePair,width,'1T',length)
                print('1 minute upper center lower: ' , bb_1m)
                # Price & Server Time
                price = client.get_ticker(symbol=tradePair)
                coitime = client.get_server_time()
                coitime = time.strftime('%m/%d/%Y %H:%M:%S',
                                        time.gmtime(coitime['serverTime']/1000.))

                # SELL 

                balance = client.get_asset_balance(asset = trdPair1)

                coiNumber = format(float(balance['free']),'.4f') 
                coiprice = format(float(price['askPrice']), '.5f')
                if (float(coiprice) > (float(lasprice) * winRate)):# and (bb_1m[0]<=float(lasprice)):
                    stat = 'sell'
                    print(stat)
                    ## order the sell comand 

                    order = client.order_limit_sell(symbol=tradePair,quantity= float(coiNumber),price= coiprice)

                    print(order)
                    lastrade = trdPair2
                    lasprice = coiprice
                else:
                    stat = 'hold' + trdPair1 + '  ' + str(lasprice*winRate)



                # Print the values
                print(coitime + ' ' + balance['free'] + '  ' + str(rsi) + '  ' + price['askPrice'] + ' ' + stat)
            except:
                print(coitime + ' ' + 'an error occured & retrying now')
                # Repeat the code every 1 minute
            time.sleep(10)

ARPA
0.04881
1 minute upper center lower:  (0.04859251347684417, 0.0482215, 0.04785048652315583)
05/27/2021 14:14:23 1024.40000000  54  0.04842000 holdARPA  0.0502743
ARPA
0.04881
1 minute upper center lower:  (0.04859251347684417, 0.0482215, 0.04785048652315583)
05/27/2021 14:14:38 1024.40000000  54  0.04842000 holdARPA  0.0502743
ARPA
0.04881
1 minute upper center lower:  (0.04859251347684417, 0.0482215, 0.04785048652315583)
05/27/2021 14:14:53 1024.40000000  54  0.04842000 holdARPA  0.0502743
ARPA
0.04881
1 minute upper center lower:  (0.04857691572036087, 0.048221, 0.04786508427963913)
05/27/2021 14:15:07 1024.40000000  49  0.04843000 holdARPA  0.0502743
ARPA
0.04881
1 minute upper center lower:  (0.048576335636214246, 0.0482205, 0.04786466436378575)
05/27/2021 14:15:23 1024.40000000  49  0.04846000 holdARPA  0.0502743
ARPA
0.04881
1 minute upper center lower:  (0.048648794228361665, 0.0482425, 0.047836205771638336)
05/27/2021 14:15:38 1024.40000000  49  0.04871000 holdARPA  0.0502

ARPA
0.04881
1 minute upper center lower:  (0.04973267203879179, 0.0487735, 0.04781432796120821)
05/27/2021 14:26:21 1024.40000000  69  0.04966000 holdARPA  0.0502743
ARPA
0.04881
1 minute upper center lower:  (0.049762954729374656, 0.048780000000000004, 0.04779704527062535)
05/27/2021 14:26:36 1024.40000000  69  0.04995000 holdARPA  0.0502743
ARPA
0.04881
1 minute upper center lower:  (0.04977782862845819, 0.048783, 0.04778817137154181)
05/27/2021 14:26:51 1024.40000000  69  0.04987000 holdARPA  0.0502743
ARPA
0.04881
1 minute upper center lower:  (0.04987346682639399, 0.048833499999999995, 0.047793533173606)
05/27/2021 14:27:05 1024.40000000  70  0.04967000 holdARPA  0.0502743
ARPA
0.04881
1 minute upper center lower:  (0.0498814918737734, 0.0488355, 0.04778950812622659)
05/27/2021 14:27:20 1024.40000000  70  0.04968000 holdARPA  0.0502743
ARPA
0.04881
1 minute upper center lower:  (0.04985454629525182, 0.0488285, 0.04780245370474817)
05/27/2021 14:27:36 1024.40000000  70  0.04957000

05/27/2021 14:38:17 1024.40000000  57  0.04940000 holdARPA  0.0502743
ARPA
0.04881
1 minute upper center lower:  (0.049798609266042196, 0.049313499999999996, 0.0488283907339578)
05/27/2021 14:38:32 1024.40000000  57  0.04940000 holdARPA  0.0502743
ARPA
0.04881
1 minute upper center lower:  (0.049801106984109465, 0.049315, 0.04882889301589053)
05/27/2021 14:38:48 1024.40000000  57  0.04950000 holdARPA  0.0502743
ARPA
0.04881
1 minute upper center lower:  (0.049810681229283046, 0.0493195, 0.04882831877071696)
05/27/2021 14:39:02 1024.40000000  57  0.04950000 holdARPA  0.0502743
ARPA
0.04881
1 minute upper center lower:  (0.04973706299736104, 0.049364000000000005, 0.04899093700263897)
05/27/2021 14:39:17 1024.40000000  59  0.04948000 holdARPA  0.0502743
ARPA
0.04881
1 minute upper center lower:  (0.04972287345181216, 0.0493545, 0.04898612654818785)
05/27/2021 14:39:31 1024.40000000  59  0.04945000 holdARPA  0.0502743
ARPA
0.04881
1 minute upper center lower:  (0.04972594564816016, 0.04935

ARPA
0.04881
1 minute upper center lower:  (0.049599722050542444, 0.049375, 0.04915027794945756)
05/27/2021 14:50:42 1024.40000000  51  0.04945000 holdARPA  0.0502743
ARPA
0.04881
1 minute upper center lower:  (0.049599722050542444, 0.049375, 0.04915027794945756)
05/27/2021 14:50:59 1024.40000000  51  0.04944000 holdARPA  0.0502743
ARPA
0.04881
1 minute upper center lower:  (0.049594798475469895, 0.049368499999999996, 0.049142201524530096)
05/27/2021 14:51:14 1024.40000000  50  0.04940000 holdARPA  0.0502743
ARPA
0.04881
1 minute upper center lower:  (0.049594798475469895, 0.049368499999999996, 0.049142201524530096)
05/27/2021 14:51:29 1024.40000000  50  0.04941000 holdARPA  0.0502743
ARPA
0.04881
1 minute upper center lower:  (0.04960525969924793, 0.049376500000000004, 0.04914774030075208)
05/27/2021 14:51:43 1024.40000000  50  0.04947000 holdARPA  0.0502743
ARPA
0.04881
1 minute upper center lower:  (0.04960525969924793, 0.049376500000000004, 0.04914774030075208)
05/27/2021 14:51:59 

05/27/2021 15:02:28 1024.40000000  46  0.04939000 holdARPA  0.0502743
ARPA
0.04881
1 minute upper center lower:  (0.049611497663538666, 0.04939750000000002, 0.04918350233646137)
05/27/2021 15:02:43 1024.40000000  46  0.04948000 holdARPA  0.0502743
ARPA
0.04881
1 minute upper center lower:  (0.049611497663538666, 0.04939750000000002, 0.04918350233646137)
05/27/2021 15:02:57 1024.40000000  46  0.04950000 holdARPA  0.0502743
ARPA
0.04881
1 minute upper center lower:  (0.04960901643129112, 0.04939600000000001, 0.0491829835687089)
05/27/2021 15:03:13 1024.40000000  51  0.04951000 holdARPA  0.0502743
ARPA
0.04881
1 minute upper center lower:  (0.04960014151800641, 0.04939250000000001, 0.04918485848199362)
05/27/2021 15:03:28 1024.40000000  51  0.04951000 holdARPA  0.0502743
ARPA
0.04881
1 minute upper center lower:  (0.049601144180797835, 0.049393000000000006, 0.04918485581920218)
05/27/2021 15:03:43 1024.40000000  51  0.04950000 holdARPA  0.0502743
ARPA
0.04881
1 minute upper center lower: 

...

0.04881
1 minute upper center lower:  (0.04760173853925892, 0.04725, 0.04689826146074108)
05/27/2021 21:50:54 1024.40000000  61  0.04738000 holdARPA  0.0502743
ARPA
0.04881
1 minute upper center lower:  (0.04760173853925892, 0.04725, 0.04689826146074108)
05/27/2021 21:51:08 1024.40000000  61  0.04738000 holdARPA  0.0502743
ARPA
0.04881
1 minute upper center lower:  (0.04758277856131736, 0.0472775, 0.04697222143868264)
05/27/2021 21:51:23 1024.40000000  63  0.04741000 holdARPA  0.0502743
ARPA
0.04881
1 minute upper center lower:  (0.04758277856131736, 0.0472775, 0.04697222143868264)
05/27/2021 21:51:37 1024.40000000  63  0.04741000 holdARPA  0.0502743
ARPA
0.04881
1 minute upper center lower:  (0.04759507774155702, 0.0472815, 0.04696792225844298)
05/27/2021 21:51:55 1024.40000000  63  0.04748000 holdARPA  0.0502743
ARPA
0.04881
1 minute upper center lower:  (0.04758518110000504, 0.047309, 0.04703281889999495)
05/27/2021 21:52:09 1024.40000000  67  0.04747000 holdARPA  0.0502743
ARPA
0.0

ARPA
0.04881
1 minute upper center lower:  (0.04752042713473726, 0.047369499999999995, 0.04721857286526273)
05/27/2021 22:02:50 1024.40000000  53  0.04737000 holdARPA  0.0502743
ARPA
0.04881
1 minute upper center lower:  (0.04752042713473726, 0.047369499999999995, 0.04721857286526273)
05/27/2021 22:03:06 1024.40000000  53  0.04737000 holdARPA  0.0502743
ARPA
0.04881
1 minute upper center lower:  (0.04751613783274781, 0.0473755, 0.047234862167252194)
05/27/2021 22:03:20 1024.40000000  57  0.04737000 holdARPA  0.0502743
ARPA
0.04881
1 minute upper center lower:  (0.04751613783274781, 0.0473755, 0.047234862167252194)
05/27/2021 22:03:35 1024.40000000  57  0.04741000 holdARPA  0.0502743
ARPA
0.04881
1 minute upper center lower:  (0.04751613783274781, 0.0473755, 0.047234862167252194)
05/27/2021 22:03:50 1024.40000000  57  0.04741000 holdARPA  0.0502743
ARPA
0.04881
1 minute upper center lower:  (0.047509917288555706, 0.0473815, 0.047253082711444294)
05/27/2021 22:04:07 1024.40000000  57  0.

05/27/2021 22:14:31 1024.40000000  48  0.04742000 holdARPA  0.0502743
ARPA
0.04881
1 minute upper center lower:  (0.04751477337503608, 0.04729150000000001, 0.04706822662496393)
05/27/2021 22:14:46 1024.40000000  48  0.04721000 holdARPA  0.0502743
ARPA
0.04881
1 minute upper center lower:  (0.04751504175558235, 0.047290500000000006, 0.047065958244417665)
05/27/2021 22:15:01 1024.40000000  48  0.04721000 holdARPA  0.0502743
ARPA
0.04881
1 minute upper center lower:  (0.047498305344138236, 0.047277500000000014, 0.04705669465586179)
05/27/2021 22:15:17 1024.40000000  47  0.04721000 holdARPA  0.0502743
ARPA
0.04881
1 minute upper center lower:  (0.04749697926168186, 0.04728000000000001, 0.04706302073831816)
05/27/2021 22:15:32 1024.40000000  47  0.04722000 holdARPA  0.0502743
ARPA
0.04881
1 minute upper center lower:  (0.047498305344138236, 0.047277500000000014, 0.04705669465586179)
05/27/2021 22:15:46 1024.40000000  47  0.04716000 holdARPA  0.0502743
ARPA
0.04881
1 minute upper center lowe

05/27/2021 22:26:16 1024.40000000  48  0.04706000 holdARPA  0.0502743
ARPA
0.04881
1 minute upper center lower:  (0.04731503967941577, 0.04706300000000001, 0.046810960320584245)
05/27/2021 22:26:31 1024.40000000  48  0.04706000 holdARPA  0.0502743
ARPA
0.04881
1 minute upper center lower:  (0.04731503967941577, 0.04706300000000001, 0.046810960320584245)
05/27/2021 22:26:46 1024.40000000  48  0.04706000 holdARPA  0.0502743
ARPA
0.04881
1 minute upper center lower:  (0.047315645912735016, 0.047056, 0.046796354087264985)
05/27/2021 22:27:01 1024.40000000  48  0.04704000 holdARPA  0.0502743
ARPA
0.04881
1 minute upper center lower:  (0.04731037851652507, 0.047046, 0.04678162148347492)
05/27/2021 22:27:15 1024.40000000  42  0.04704000 holdARPA  0.0502743
ARPA
0.04881
1 minute upper center lower:  (0.04731037851652507, 0.047046, 0.04678162148347492)
05/27/2021 22:27:31 1024.40000000  42  0.04704000 holdARPA  0.0502743
ARPA
0.04881
1 minute upper center lower:  (0.04732909699737223, 0.0470610

KeyboardInterrupt: 